# 特定の日時のリストを取得する 

In [1]:
import jpholiday
from pytz import timezone, utc
import datetime
import pickle
from pathlib import Path

In [2]:
import numpy as np
import pandas as pd

In [3]:
def check_jst_datetimes_to_naive(*arg_datetimes):
    """
    ＊*今のところ，ローカルが東京でないnaiveなdatetimeはそのまま通してしまう
    引数のタイムゾーンが同じかどうかチェックし，存在するなら日本であるかチェック
    awareな場合は，naiveに変更
    """
    jst_timezone = timezone("Asia/Tokyo")
    tz_info_set = set([one_datetime.tzinfo for one_datetime in arg_datetimes])
    if len(tz_info_set) > 1:
        raise Exception("timezones are different")
        
    datetimes_tzinfo = list(tz_info_set)[0]
    
    if datetimes_tzinfo is not None:  # 長さが1のはず
        if timezone(str(datetimes_tzinfo)) != jst_timezone:
            raise Exception("timezones must be Asia/Tokyo")
        # naiveなdatetimeに変更
        arg_datetimes = [one_datetime.replace(tzinfo=None) for one_datetime in arg_datetimes]
    
    # 引数が一つであるかどうか
    if len(arg_datetimes) > 1:  # 引数が複数の場合
        return tuple(arg_datetimes)
    else:  # 引数が一つの場合
        return arg_datetimes[0]

## 営業日・休日を取得する 

Optionの簡単化のため，HolidayGetterクラスを作成する．jpholidayを利用するものとCSVファイルを読み込むものの二つを用意する．

### 祝日のndarrayを取得する(jpholidayを利用)

以下はjpholidayを利用した素直な実装だが，遅い．そこで一度しか呼ばないようにすべき

In [4]:
class JPHolidayGetter:
    def get_holidays(self, start_date, end_date, with_name=False):
        """
        期間を指定して祝日を取得．jpholidayを利用して祝日を取得している．

        start_date: datetime.date
            開始時刻のdate
        end_datetime: datetime.date
            終了時刻のdate
        with_name: bool
            休日の名前を出力するかどうか
        to_date: bool
            出力をdatetime.datetimeにするかdatetime.dateにするか
        """
        assert isinstance(start_date, datetime.date) and isinstance(end_date, datetime.date)

        holidays_array = np.array(jpholiday.between(start_date, end_date))

        if not with_name:  # 祝日名がいらない場合
            return holidays_array[:,0].copy()

        return holidays_array
        

In [5]:
start_date = datetime.date(1960, 1, 1)
end_date = datetime.date(2021, 12, 31)

holidays_array = JPHolidayGetter().get_holidays(start_date, end_date, with_name=True)

In [6]:
holidays_array

array([[datetime.date(1960, 1, 1), '元日'],
       [datetime.date(1960, 1, 15), '成人の日'],
       [datetime.date(1960, 3, 20), '春分の日'],
       ...,
       [datetime.date(2021, 9, 23), '秋分の日'],
       [datetime.date(2021, 11, 3), '文化の日'],
       [datetime.date(2021, 11, 23), '勤労感謝の日']], dtype=object)

In [7]:
len(holidays_array)

937

### 祝日のndarrayを取得(CSVファイルを利用) 

思ったより速度が速いので，毎回これを呼ぶことも可能であるが，jpholidayのものを踏襲してこれも一度あるいはOptionを変更したときのみ呼ばれるようにする．

In [8]:
class CSVHolidayGetter:
    def __init__(self, csv_paths):
        if not isinstance(csv_paths, list):  # リストでないなら，リストにしておく
            csv_paths = [csv_paths]
            
        self.csv_paths = csv_paths
        
    def get_holidays(self, start_date, end_date, with_name=False):
        """
        期間を指定して祝日を取得．csvファイルを利用して祝日を取得している．

        start_date: datetime.date
            開始時刻のdate
        end_datetime: datetime.date
            終了時刻のdate
        with_name: bool
            休日の名前を出力するかどうか
        to_date: bool
            出力をdatetime.datetimeにするかdatetime.dateにするか
        """
        assert isinstance(start_date, datetime.date) and isinstance(end_date, datetime.date)
        
        # datetime.dateをpd.Timestampに変換(datetime.dateは通常pd.DatetimeIndexと比較できないため)
        start_timestamp = pd.Timestamp(start_date)
        end_timestamp = pd.Timestamp(end_date)
        
        for i, csv_path in enumerate(self.csv_paths):
            holiday_df = pd.read_csv(csv_path, 
                                     header=None,
                                     names=["date", "holiday_name"],
                                     index_col="date",
                                     parse_dates=True
                                    )
            if i == 0:
                left_df = holiday_df
            else:
                append_bool = ~holiday_df.index.isin(left_df.index)  # 左Dataframeに存在しない部分を追加
                left_df = left_df.append(holiday_df.loc[append_bool], sort=True)

        
        # 指定範囲内の祝日を取得
        holiday_in_span_index = (start_timestamp<=left_df.index)&(left_df.index<end_timestamp)
        holiday_in_span_df = left_df.loc[holiday_in_span_index]
        
        holiday_in_span_date_array = holiday_in_span_df.index.date
        holiday_in_span_name_array = holiday_in_span_df.loc[:,"holiday_name"].values
        holiday_in_span_array = np.stack([holiday_in_span_date_array,
                                          holiday_in_span_name_array
                                         ],
                                         axis=1
                                        )
        
        if not with_name:  # 祝日名がいらない場合
            return holiday_in_span_date_array
            
        return holiday_in_span_array
            

In [9]:
source_paths = [Path("source/holiday_naikaku.csv"),
                Path("source/holiday_api.csv")
               ]

holiday_getter = CSVHolidayGetter(source_paths)

start_date = datetime.date(1960, 1, 1)
end_date = datetime.date(2021, 12, 31)

holidays_array = holiday_getter.get_holidays(start_date, end_date, with_name=True)
holidays_array

array([[datetime.date(1960, 1, 1), '元日'],
       [datetime.date(1960, 1, 15), '成人の日'],
       [datetime.date(1960, 3, 20), '春分の日'],
       ...,
       [datetime.date(2021, 9, 23), '秋分の日'],
       [datetime.date(2021, 11, 3), '文化の日'],
       [datetime.date(2021, 11, 23), '勤労感謝の日']], dtype=object)

In [10]:
len(holidays_array)

913

## Option

jpholidayの休日取得速度が遅いため(csvとしても毎回読み込むのは非効率なため)，パッケージとして読み込む時にあらかじめ利用する休日をarrayとして保持しておくOptionを作成する．さらに利用する休日の期間やcsvとjpholidayの切り替え，休日曜日の切り替え，日中時間の切り替えなどを行うことができる．

In [11]:
class Option():
    """
    オプションの指定のためのクラス
    holiday_start_year: int
        利用する休日の開始年
    holiday_end_year: int
        利用する休日の終了年
    backend: str
        休日取得のバックエンド．csvかjpholidayのいずれかが選べる
    csv_source_paths: list of str or pathlib.Path
        バックエンドをcsvにした場合の休日のソースcsvファイル
    holiday_weekdays: list of int
        休日曜日の整数のリスト
    intraday_borders: list of list of 2 datetime.time
        日中を指定する境界時間のリストのリスト
    """
    def __init__(self):
        self._holiday_start_year = datetime.datetime.now().year-5
        self._holiday_end_year = datetime.datetime.now().year
        
        self._backend = "csv"
        self._csv_source_paths = [Path("source/holiday_naikaku.csv"),]
        
        self.make_holiday_getter()  # HolidayGetterを作成
        self.make_holidays()  # アトリビュートに追加
        
        self._holiday_weekdays = [5,6]  # 土曜日・日曜日
        self._intraday_borders = [[datetime.time(9,0), datetime.time(11,30)],
                                  [datetime.time(12,30), datetime.time(15,0)]]
        
    
    def make_holiday_getter(self):
        if self.backend == "jp_holiday":
            self._holiday_getter = JPHolidayGetter()
        elif self.backend == "csv":
            self._holiday_getter = CSVHolidayGetter(self.csv_source_paths)
        
    
    def make_holidays(self):
        """
        利用する休日のarrayとDatetimeIndexをアトリビュートとして作成
        """
        self._holidays_date_array = self._holiday_getter.get_holidays(start_date=datetime.date(self.holiday_start_year,1,1),
                                                    end_date=datetime.date(self.holiday_end_year,12,31),
                                                    with_name=False,
                                                   )
        self._holidays_datetimeindex =  pd.DatetimeIndex(self._holidays_date_array)
        
    
    @property
    def holiday_start_year(self):
        return self._holiday_start_year
    
    @holiday_start_year.setter
    def holiday_start_year(self, year):
        assert isinstance(year,int)
        self._holiday_start_year = year
        self.make_holidays()  # アトリビュートに追加
    
    @property
    def holiday_end_year(self):
        return self._holiday_end_year

    @holiday_end_year.setter
    def holiday_end_year(self, year):
        assert isinstance(year,int)
        self._holiday_end_year = year
        self.make_holidays()  # アトリビュートに追加
    
    @property
    def backend(self):
        return self._backend
    
    @backend.setter
    def backend(self, backend_str):
        if backend_str not in ("jp_holiday","csv"):
            raise Exception("backend must be 'jp_holiday' or 'csv'.")
        self._backend = backend_str
        self.make_holiday_getter()  # HolidayGetterを作成
    
    @property
    def csv_source_paths(self):
        #中身の確認
        for csv_source_path in self._csv_source_paths:
            if not isinstance(csv_source_path, str) and not isinstance(csv_source_path, Path):
                raise Exception("csv_source_paths must be list of str or pathlib.Path")
        
        return self._csv_source_paths
    
    @csv_source_paths.setter
    def csv_source_paths(self, path_list):
        self.csv_source_paths = path_list
        
    @property
    def holidays_date_array(self):
        return self._holidays_date_array
    
    @property
    def holidays_datetimeindex(self):
        return self._holidays_datetimeindex
    
    @property
    def holiday_weekdays(self):
        # 中身の確認
        for weekday in self._holiday_weekdays:
            if not isinstance(weekday, int):
                raise Exception("holiday_weekdays must be list of integer(0<=x<=6)")
                
        return self._holiday_weekdays
    
    @holiday_weekdays.setter
    def holiday_weekdays(self, weekdays_list):
        self._holiday_weekdays = weekdays_list
    
    @property
    def intraday_borders(self):
        # 中身の確認
        for border in self._intraday_borders:
            if not isinstance(border, list) or len(border) != 2:
                raise Exception("intraday_borders must be list of list whitch has 2 datetime.time")
                
            for border_time in border:
                if not isinstance(border_time, datetime.time):
                    raise Exception("intraday_borders must be list of list whitch has 2 datetime.time")
             
        return self._intraday_borders
    
    @intraday_borders.setter
    def intraday_borders(self, borders_list):
        self._intraday_borders = borders_list

### Optionのコンストラクト 

In [12]:
option = Option()

### 休日を取得する関数

In [13]:
def get_holidays_jp(start_date, end_date, with_name=False, independent=False):
        """
        期間を指定して祝日を取得．

        start_date: datetime.date
            開始時刻のdate
        end_datetime: datetime.date
            終了時刻のdate
        with_name: bool
            休日の名前を出力するかどうか
        to_date: bool
            出力をdatetime.datetimeにするかdatetime.dateにするか
        independent: bool，default:False
            休日をoptionから独立させるかどうか．FalseならばOption内で保持する休日が取得される
        """
        assert isinstance(start_date, datetime.date) and isinstance(end_date, datetime.date)
        
        if not independent:
            # datetime.dateをpd.Timestampに変換(datetime.dateは通常pd.DatetimeIndexと比較できないため)
            start_timestamp = pd.Timestamp(start_date)
            end_timestamp = pd.Timestamp(end_date)

            holidays_in_span_index = (start_timestamp<=option.holidays_datetimeindex)&(option.holidays_datetimeindex<end_timestamp)  # DatetimeIndexを使うことに注意
            holidays_in_span_array = option.holidays_date_array[holidays_in_span_index]  # ndarrayを使うtamp(end_date)

            return holidays_in_span_array
        else:
            if option.backend == "jp_holiday":
                holiday_getter = JPHolidayGetter()
            elif option.backend == "csv":
                holiday_getter = CSVHolidayGetter(option.csv_source_paths)
                
            holidays_array = holiday_getter.get_holidays(start_date=start_date,
                                                         end_date=end_date,
                                                         with_name=with_name
                                                        ) 
            return  holidays_array

In [14]:
start_date = datetime.date(1960, 1, 1)
end_date = datetime.date(2021, 12, 31)

holidays_array = get_holidays_jp(start_date, 
                                end_date,
                                with_name=True,
                                independent=False
                               )
holidays_array

array([datetime.date(2016, 1, 1), datetime.date(2016, 1, 11),
       datetime.date(2016, 2, 11), datetime.date(2016, 3, 20),
       datetime.date(2016, 3, 21), datetime.date(2016, 4, 29),
       datetime.date(2016, 5, 3), datetime.date(2016, 5, 4),
       datetime.date(2016, 5, 5), datetime.date(2016, 7, 18),
       datetime.date(2016, 8, 11), datetime.date(2016, 9, 19),
       datetime.date(2016, 9, 22), datetime.date(2016, 10, 10),
       datetime.date(2016, 11, 3), datetime.date(2016, 11, 23),
       datetime.date(2016, 12, 23), datetime.date(2017, 1, 1),
       datetime.date(2017, 1, 2), datetime.date(2017, 1, 9),
       datetime.date(2017, 2, 11), datetime.date(2017, 3, 20),
       datetime.date(2017, 4, 29), datetime.date(2017, 5, 3),
       datetime.date(2017, 5, 4), datetime.date(2017, 5, 5),
       datetime.date(2017, 7, 17), datetime.date(2017, 8, 11),
       datetime.date(2017, 9, 18), datetime.date(2017, 9, 23),
       datetime.date(2017, 10, 9), datetime.date(2017, 11, 3),

## 営業日の取得 

optionの休日を参照する．

### 指定した期間の営業日のndarrayを取得する

In [15]:
def get_workdays_jp(start_date, end_date, return_as="date", end_include=False):
    """
    営業日を取得
    
    start_date: datetime.date
        開始時刻のdate
    end_datetime: datetime.date
        終了時刻のdate
    return_as: str, defalt: 'dt'
        返り値の形式
        - 'dt':pd.DatetimeIndex
        - 'date': datetime.date array
    end_include: bool
        最終日も含めて出力するか
    """
    assert isinstance(start_date, datetime.date) and isinstance(end_date, datetime.date)
    # 返り値の形式の指定
    return_as_set = {"dt", "date"}
    if not return_as in return_as_set:
        raise Exception("return_as must be any in {}".format(return_as_set))
    
    # datetime.dateをpd.Timestampに変換(datetime.dateは通常pd.DatetimeIndexと比較できないため)
    start_timestamp = pd.Timestamp(start_date)
    end_timestamp = pd.Timestamp(end_date)
    
    # 期間中のholidayを取得
    holidays_in_span_index = (start_timestamp<=option.holidays_datetimeindex)&(option.holidays_datetimeindex<end_timestamp)  # DatetimeIndexを使うことに注意
    holidays_in_span_array = option.holidays_date_array[holidays_in_span_index]  # ndarrayを使う

    # 期間中のdatetimeのarrayを取得
    if end_include:
        days_datetimeindex = pd.date_range(start=start_date, end=end_date, freq="D")  # 最終日も含める
    else:
        days_datetimeindex = pd.date_range(start=start_date, end=end_date-datetime.timedelta(days=1), freq="D")  # 最終日は含めない
    
    
    # 休日に含まれないもの，さらに土日に含まれないもののboolインデックスを取得
    holiday_bool_array = np.in1d(days_datetimeindex.date, holidays_in_span_array)  # 休日であるかのブール(pd.DatetimeIndex.isin)でもいい
    holiday_weekday_each_bool_arrays = [days_datetimeindex.weekday==weekday for weekday in option.holiday_weekdays]  # inを使うのを回避
    holiday_weekday_bool_array = np.logical_or.reduce(holiday_weekday_each_bool_arrays)  # 休日曜日
    
    workdays_bool_array = (~holiday_bool_array)&(~holiday_weekday_bool_array)  # 休日でなく休日曜日でない
    
    workdays_datetimeindex = days_datetimeindex[workdays_bool_array].copy()
    if return_as=="dt":
        return workdays_datetimeindex
    elif return_as=="date":
        return workdays_datetimeindex.date

In [16]:
start_date = datetime.datetime(2021, 1, 1)
end_date = datetime.datetime(2021, 12, 31)

workdays = get_workdays_jp(start_date, end_date, return_as="date", end_include=False)

In [17]:
workdays[:10]

array([datetime.date(2021, 1, 4), datetime.date(2021, 1, 5),
       datetime.date(2021, 1, 6), datetime.date(2021, 1, 7),
       datetime.date(2021, 1, 8), datetime.date(2021, 1, 12),
       datetime.date(2021, 1, 13), datetime.date(2021, 1, 14),
       datetime.date(2021, 1, 15), datetime.date(2021, 1, 18)],
      dtype=object)

### 営業日であるかどうか 

In [18]:
def check_workday_jp(select_date):
    """
    与えられたdatetime.dateが営業日であるかどうかを出力する
    select_date: datetime.date
        入力するdate
    """
    assert isinstance(select_date, datetime.date)
    select_date_array = np.array([select_date])  # データ数が一つのndarray
    # 休日であるかどうか
    is_holiday = np.in1d(select_date_array, option.holidays_date_array).item()  # データ数が一つのため，item
    
    # 休日曜日であるかどうか
    is_holiday_weekday = any([select_date.weekday()==weekday for weekday in option.holiday_weekdays])
    
    is_workday = (not is_holiday) and (not is_holiday_weekday)
    
    return is_workday

In [19]:
select_date = datetime.date(2021, 1, 3)
check_workday_jp(select_date)

False

### 非営業日を取得

同様に，すでに取得したholidayと照らし合わせる．

In [20]:
def get_not_workdays_jp(start_date, end_date, return_as="date", end_include=False):
    """
    非営業日を取得(土日or祝日)
    
    start_date: datetime.date
        開始時刻のdate
    end_datetime: datetime.date
        終了時刻のdate
    return_as: str, defalt: 'dt'
        返り値の形式
        - 'dt':pd.DatetimeIndex
        - 'date': datetime.date array
    end_include: bool
        最終日も含めて出力するか
    """
    assert isinstance(start_date, datetime.date) and isinstance(end_date, datetime.date)
    # 返り値の形式の指定
    return_as_set = {"dt", "date"}
    if not return_as in return_as_set:
        raise Exception("return_as must be any in {}".format(return_as_set))
    
    # datetime.dateをpd.Timestampに変換(datetime.dateは通常pd.DatetimeIndexと比較できないため)
    start_timestamp = pd.Timestamp(start_date)
    end_timestamp = pd.Timestamp(end_date)
    
    # 期間中のholidayを取得
    holidays_in_span_index = (start_timestamp<=option.holidays_datetimeindex)&(option.holidays_datetimeindex<end_timestamp)  # DatetimeIndexを使うことに注意
    holidays_in_span_array = option.holidays_date_array[holidays_in_span_index]  # ndarrayを使う

    # 期間中のdatetimeのarrayを取得
    if end_include:
        days_datetimeindex = pd.date_range(start=start_date, end=end_date, freq="D")  # 最終日も含める
    else:
        days_datetimeindex = pd.date_range(start=start_date, end=end_date-datetime.timedelta(days=1), freq="D")  # 最終日は含めない
    
    # 休日に含まれないもの，さらに休日曜日に含まれないもののboolインデックスを取得
    holiday_bool_array = np.in1d(days_datetimeindex.date, holidays_in_span_array)  # 休日であるかのブール(pd.DatetimeIndex.isin)でもいい
    holiday_weekday_each_bool_arrays = [days_datetimeindex.weekday==weekday for weekday in option.holiday_weekdays]  # inを使うのを回避
    holiday_weekday_bool_array = np.logical_or.reduce(holiday_weekday_each_bool_arrays)  # 休日曜日
    
    not_workdays_bool_array = holiday_bool_array | holiday_weekday_bool_array  # 休日あるいは休日曜日
    
    not_workdays_datetimeindex = days_datetimeindex[not_workdays_bool_array].copy()
    if return_as=="dt":
        return not_workdays_datetimeindex
    elif return_as=="date":
        return not_workdays_datetimeindex.date

In [21]:
start_datetime = datetime.date(2019, 1, 1)
end_datetime = datetime.date(2019, 12, 31)

not_workdays = get_not_workdays_jp(start_date, end_date, return_as="dt")

In [22]:
not_workdays

DatetimeIndex(['2021-01-01', '2021-01-02', '2021-01-03', '2021-01-09',
               '2021-01-10', '2021-01-11', '2021-01-16', '2021-01-17',
               '2021-01-23', '2021-01-24',
               ...
               '2021-11-27', '2021-11-28', '2021-12-04', '2021-12-05',
               '2021-12-11', '2021-12-12', '2021-12-18', '2021-12-19',
               '2021-12-25', '2021-12-26'],
              dtype='datetime64[ns]', length=119, freq=None)

## 指定した日数後の営業日を取得

ジェネレーターを使うので速度注意

In [23]:
def get_next_workday_jp(select_date, return_as="date", days=1):
    """
    select_date: datetime.date
        指定する日時
    return_as: str, defalt: 'dt'
        返り値の形式
        - 'dt':pd.Timstamp
        - 'datetime': datetime.datetime array
    """
    assert isinstance(select_date, datetime.date)
    # 返り値の形式の指定
    return_as_set = {"dt", "date"}
    if not return_as in return_as_set:
        raise Exception("return_as must be any in {}".format(return_as_set))
        
    def get_next_workday_jp_gen(select_date):
        add_days=1
        while True:
            next_day = select_date + datetime.timedelta(days=add_days)
            if check_workday_jp(next_day):
                yield next_day
            add_days += 1
    
    next_workday_gen = get_next_workday_jp_gen(select_date)
    for i in range(days):
        next_day = next(next_workday_gen)
    
    if return_as=="date":
        return next_day
    elif return_as=="dt":
        return pd.Timestamp(next_day)

In [24]:
jst_timezone = timezone("Asia/Tokyo")
select_datetime = jst_timezone.localize(datetime.datetime(2021, 1, 1, 0, 0, 0))

next_workday = get_next_workday_jp(select_datetime, days=6, return_as="dt")
next_workday

Timestamp('2021-01-11 00:00:00+0900', tz='Asia/Tokyo')

## データフレームに関しての処理

### データの用意 

awareなdatetimeを持つDataFrame

In [25]:
with open("aware_stock_df.pickle", "rb") as f:
    aware_stock_df = pickle.load(f)

aware_stock_df

,Open_6502,High_6502,Low_6502,Close_6502,Volume_6502
timestamp,,,,,
2020-11-04 00:00:00+09:00,NaN,NaN,NaN,NaN,NaN
2020-11-04 00:01:00+09:00,NaN,NaN,NaN,NaN,NaN
2020-11-04 00:02:00+09:00,NaN,NaN,NaN,NaN,NaN
2020-11-04 00:03:00+09:00,NaN,NaN,NaN,NaN,NaN
2020-11-04 00:04:00+09:00,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...
2020-11-30 23:55:00+09:00,NaN,NaN,NaN,NaN,NaN
2020-11-30 23:56:00+09:00,NaN,NaN,NaN,NaN,NaN
2020-11-30 23:57:00+09:00,NaN,NaN,NaN,NaN,NaN


naiveなdatetimeを持つDataFrame

In [26]:
with open("naive_stock_df.pickle", "rb") as f:
    naive_stock_df = pickle.load(f)

naive_stock_df.at_time(datetime.time(9,0))

,Open_6502,High_6502,Low_6502,Close_6502,Volume_6502
timestamp,,,,,
2020-11-04 09:00:00,2669.0,2670.0,2658.0,2664.0,93000.0
2020-11-05 09:00:00,2631.0,2641.0,2630.0,2639.0,55000.0
2020-11-06 09:00:00,2615.0,2644.0,2609.0,2617.0,90700.0
2020-11-09 09:00:00,2711.0,2719.0,2707.0,2710.0,0.0
2020-11-10 09:00:00,2780.0,2782.0,2770.0,2777.0,142700.0
2020-11-11 09:00:00,NaN,NaN,NaN,NaN,NaN
2020-11-12 09:00:00,2788.0,2800.0,2781.0,2789.0,108400.0
2020-11-13 09:00:00,2750.0,2754.0,2741.0,2750.0,71500.0
2020-11-14 09:00:00,NaN,NaN,NaN,NaN,NaN


### 営業日データを取得

高速化のため，既存のholidayと照らし合わせる方法でとる

In [27]:
def extract_workdays_jp_index(dt_index, return_as="index"):
    """
    pd.DatetimeIndexから，営業日のデータのものを抽出
    dt_index: pd.DatetimeIndex
        入力するDatetimeIndex，すでにdatetimeでソートしていることが前提
    return_as: str
        出力データの形式
        - "index": 引数としたdfの対応するインデックスを返す
        - "bool": 引数としたdfに対応するboolインデックスを返す
    """
    # 返り値の形式の指定
    return_as_set = {"index", "bool"}
    if not return_as in return_as_set:
        raise Exception("return_as must be any in {}".format(return_as_set))
        
    # すでにtimestampでソートさてている前提
    start_datetime = dt_index[0].to_pydatetime()
    end_datetime = dt_index[-1].to_pydatetime()
    
    start_datetime, end_datetime = check_jst_datetimes_to_naive(start_datetime, end_datetime)  # 二つのdatetimeのタイムゾーンをチェック・naiveに変更
    
    # 期間内のholidayを取得
    holidays_in_span_index = ((start_datetime-datetime.timedelta(days=1))<option.holidays_datetimeindex)&\
    (option.holidays_datetimeindex<=end_datetime)  # DatetimeIndexを使うことに注意, 当日を含めるため，startから1を引いている．
    holidays_in_span_array = option.holidays_date_array[holidays_in_span_index]  # ndarrayを使う
    
    # 休日に含まれないもの，さらに土日に含まれないもののboolインデックスを取得    
    holiday_bool_array = np.in1d(dt_index.date, holidays_in_span_array)  # 休日
    holiday_weekday_each_bool_arrays = [dt_index.weekday==weekday for weekday in option.holiday_weekdays]  # inを使うのを回避
    holiday_weekday_bool_array = np.logical_or.reduce(holiday_weekday_each_bool_arrays)  # 休日曜日
    
    workdays_bool_array = (~holiday_bool_array)&(~holiday_weekday_bool_array)  # 休日でなく休日曜日でない
    if return_as=="bool":  # boolで返す場合
        return workdays_bool_array
    
    elif return_as=="index":  # indexで返す場合
        workdays_df_indice = dt_index[workdays_bool_array]
        return workdays_df_indice

In [28]:
def extract_workdays_jp(df, return_as="df"):
    """
    データフレームから，営業日のデータのものを抽出．出力データ形式をreturn_asで指定する．
    df: pd.DataFrame(インデックスとしてpd.DatetimeIndex)
        入力データ
    return_as: str
        出力データの形式
        - "df": 抽出した新しいpd.DataFrameを返す
        - "index": 引数としたdfの対応するインデックスを返す
        - "bool": 引数としたdfに対応するboolインデックスを返す
    """
    
    # 返り値の形式の指定
    return_as_set = {"df", "index", "bool"}
    if not return_as in return_as_set:
        raise Exception("return_as must be any in {}".format(return_as_set))
    
    workdays_bool_array = extract_workdays_jp_index(df.index, return_as="bool")
    if return_as=="bool":
        return workdays_bool_array
    
    workdays_df_indice = df.index[workdays_bool_array]
    if return_as=="index":
        return workdays_df_indice

    out_df = df.loc[workdays_df_indice].copy()
    return out_df

In [29]:
extracted_stock_df = extract_workdays_jp(aware_stock_df, return_as="df")
extracted_stock_df.at_time(datetime.time(9,0))

,Open_6502,High_6502,Low_6502,Close_6502,Volume_6502
timestamp,,,,,
2020-11-04 09:00:00+09:00,2669.0,2670.0,2658.0,2664.0,93000.0
2020-11-05 09:00:00+09:00,2631.0,2641.0,2630.0,2639.0,55000.0
2020-11-06 09:00:00+09:00,2615.0,2644.0,2609.0,2617.0,90700.0
2020-11-09 09:00:00+09:00,2711.0,2719.0,2707.0,2710.0,0.0
2020-11-10 09:00:00+09:00,2780.0,2782.0,2770.0,2777.0,142700.0
2020-11-11 09:00:00+09:00,NaN,NaN,NaN,NaN,NaN
2020-11-12 09:00:00+09:00,2788.0,2800.0,2781.0,2789.0,108400.0
2020-11-13 09:00:00+09:00,2750.0,2754.0,2741.0,2750.0,71500.0
2020-11-16 09:00:00+09:00,2714.0,2716.0,2705.0,2710.0,111000.0


### 日中データを取得

おそらく使わない，以下の営業日＋日中内で使う．

In [30]:
def extract_intraday_jp_index(dt_index, return_as="index"):
    """
    pd.DatetimeIndexから，日中(9時から11時半，12時半から15時)のデータのものを抽出．出力データ形式をreturn_asで指定する．
    dt_index: pd.DatetimeIndex
        入力するDatetimeIndex
    return_as: str
        出力データの形式
        - "index": 引数としたdfの対応するインデックスを返す
        - "bool": 引数としたdfに対応するboolインデックスを返す
    """
    
    # 返り値の形式の指定
    return_as_set = {"index", "bool"}
    if not return_as in return_as_set:
        raise Exception("return_as must be any in {}".format(return_as_set))    
  
    bool_array = np.array([False]*len(dt_index))
    
    # ボーダー内のboolをTrueにする
    for borders in option.intraday_borders:
        start_time, end_time = borders[0], borders[1]  # 開始時刻と終了時刻
        in_border_indice = dt_index.indexer_between_time(start_time=start_time, end_time=end_time, include_end=False)
        bool_array[in_border_indice] = True
    
    if return_as=="bool":
        return bool_array

    elif return_as=="index":
        intraday_indice = dt_index[bool_array]
        return intraday_indice

In [31]:
def extract_intraday_jp(df, return_as="df"):
    """
    データフレームから，日中(9時から11時半，12時半から15時)のデータのものを抽出．出力データ形式をreturn_asで指定する．
    df: pd.DataFrame(インデックスとしてpd.DatetimeIndex)
        入力データ
    return_as: str
        出力データの形式
        - "df": 抽出した新しいpd.DataFrameを返す
        - "index": 引数としたdfの対応するインデックスを返す
        - "bool": 引数としたdfに対応するboolインデックスを返す
    """
    
    # 返り値の形式の指定
    return_as_set = {"df", "index", "bool"}
    if not return_as in return_as_set:
        raise Exception("return_as must be any in {}".format(return_as_set))    
  
    intraday_bool_array = extract_intraday_jp_index(df.index, return_as="bool")
    if return_as=="bool":
        return intraday_bool_array
    
    intraday_indice = df.index[intraday_bool_array]
    if return_as=="index":
        return intraday_indice
    
    out_df = df.loc[intraday_indice].copy()
    return out_df

以下の例では，朝8時のデータは存在しない

In [32]:
extracted_df = extract_intraday_jp(naive_stock_df, return_as="df")
extracted_df.at_time(datetime.time(8,0))

,Open_6502,High_6502,Low_6502,Close_6502,Volume_6502
timestamp,,,,,


### 営業日＋日中データを取得 

In [33]:
def extract_workdays_intraday_jp_index(dt_index, return_as="index"):
    """
    pd.DatetimeIndexから，営業日+日中(9時から11時半，12時半から15時)のデータのものを抽出．出力データ形式をreturn_asで指定する．
    dt_index: pd.DatetimeIndex
        入力するDatetimeIndex
    return_as: str
        出力データの形式
        - "index": 引数としたdfの対応するインデックスを返す
        - "bool": 引数としたdfに対応するboolインデックスを返す
    """

    # 返り値の形式の指定
    return_as_set = {"index", "bool"}
    if not return_as in return_as_set:
        raise Exception("return_as must be any in {}".format(return_as_set))
        
    workday_bool_array = extract_workdays_jp_index(dt_index, return_as="bool")
    intraday_bool_array = extract_intraday_jp_index(dt_index, return_as="bool")
    
    workday_intraday_bool_array = workday_bool_array & intraday_bool_array
    if return_as=="bool":
        return workday_intraday_bool_array
    elif return_as=="index":
        workday_intraday_indice = dt_index[workday_intraday_bool_array]
        return workday_intraday_indice

In [34]:
def extract_workdays_intraday_jp(df, return_as="df"):
    """
    データフレームから，営業日+日中(9時から11時半，12時半から15時)のデータのものを抽出．出力データ形式をreturn_asで指定する．
    df: pd.DataFrame(インデックスとしてpd.DatetimeIndex)
        入力データ
    return_as: str
        出力データの形式
        - "df": 抽出した新しいpd.DataFrameを返す
        - "index": 引数としたdfの対応するインデックスを返す
        - "bool": 引数としたdfに対応するboolインデックスを返す
    """
    
    # 返り値の形式の指定
    return_as_set = {"df", "index", "bool"}
    if not return_as in return_as_set:
        raise Exception("return_as must be any in {}".format(return_as_set))    
       
    workday_intraday_bool_array = extract_workdays_intraday_jp_index(df.index, return_as="bool")
    
    if return_as=="bool":
        return workday_intraday_bool_array
    
    workday_intraday_indice = df.index[workday_intraday_bool_array]
    
    if return_as=="index":
        return workday_intraday_indice
    
    out_df = df.loc[workday_intraday_indice].copy()
    return out_df

In [35]:
extracted_df = extract_workdays_intraday_jp(aware_stock_df, return_as="df")
extracted_df.at_time(datetime.time(9,0))

,Open_6502,High_6502,Low_6502,Close_6502,Volume_6502
timestamp,,,,,
2020-11-04 09:00:00+09:00,2669.0,2670.0,2658.0,2664.0,93000.0
2020-11-05 09:00:00+09:00,2631.0,2641.0,2630.0,2639.0,55000.0
2020-11-06 09:00:00+09:00,2615.0,2644.0,2609.0,2617.0,90700.0
2020-11-09 09:00:00+09:00,2711.0,2719.0,2707.0,2710.0,0.0
2020-11-10 09:00:00+09:00,2780.0,2782.0,2770.0,2777.0,142700.0
2020-11-11 09:00:00+09:00,NaN,NaN,NaN,NaN,NaN
2020-11-12 09:00:00+09:00,2788.0,2800.0,2781.0,2789.0,108400.0
2020-11-13 09:00:00+09:00,2750.0,2754.0,2741.0,2750.0,71500.0
2020-11-16 09:00:00+09:00,2714.0,2716.0,2705.0,2710.0,111000.0


In [36]:
extracted_df.at_time(datetime.time(8,0))

,Open_6502,High_6502,Low_6502,Close_6502,Volume_6502
timestamp,,,,,
